Результат работы генератора:

{'mail': 'volyntsev.da@phystech.edu',
 'id': 625885368,
 'type': 'classification',
 'dataset': {'name': 'MNIST', 'url': 'http://yann.lecun.com/exdb/mnist/'},
 'method': ['Логистическая регрессия', 'Перцептрон']}

In [57]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [58]:
from torch import nn, optim

In [59]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

Конвертация картинки в числа и их нормализация, получаем числовое признаковое описание

In [60]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])

Загрузим датасет MNIST и получим тренировочную и тестовую выборки

In [61]:
train_dataset = MNIST('mnist', download=True, train=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = MNIST('mnist', download=True, train=False, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [62]:
images, labels = iter(train_loader).next()

print(images.shape)
print(labels.shape) #количество картинок = размеру батча

torch.Size([64, 1, 28, 28])
torch.Size([64])


Картинки размера 28 на 28 с различной степенью интенсивности цвета (0, 255) от белого к черному, признаки вещественные, всего 28*28=784 числа

In [63]:
train_data = TensorDataset(train_dataset.train_data.view(-1, 28 * 28).float() / 255)
test_data = TensorDataset(test_dataset.test_data.view(-1, 28 * 28).float() / 255)

print(train_data[0])

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000,

Инициализируем нейросеть

## LogReg

In [64]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [65]:
def learning(criterion, optimizer):
    it = 0
    accuracy = 0
    for epoch in range(int(epochs)):
        for i, (images, labels) in enumerate(train_loader):
            images = torch.autograd.Variable(images.view(-1, 28 * 28))
            labels = torch.autograd.Variable(labels)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
            # считаем accuracy
            it += 1
            if it % 500 == 0:
                correct = 0
                total = 0
                for images, labels in test_loader:
                    images = torch.autograd.Variable(images.view(-1, 28*28))
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum()
                accuracy = 100 * correct/total
            
    print("Loss: {}. Accuracy: {}.".format(loss.item(), accuracy))

In [66]:
batch_size = 64
iterations = 5000
epochs = iterations / (len(train_dataset) / batch_size)

model = LogisticRegression(784, 10) #сразу зададим размерности входа(28*28) и выхода(10 цифр)

Смотрим для разных гиперпараметров

In [67]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum = 0.9)
learning(criterion, optimizer)

Loss: 0.40412986278533936. Accuracy: 91.


In [68]:
batch_size = 32
iterations = 4000
epochs = iterations / (len(train_dataset) / batch_size)

In [69]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
learning(criterion, optimizer)

Loss: 0.22745150327682495. Accuracy: 92.


In [70]:
batch_size = 128
iterations = 3000
epochs = iterations / (len(train_dataset) / batch_size)

In [71]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.001)
learning(criterion, optimizer)

Loss: 0.3111974895000458. Accuracy: 92.


## Perceptron

In [80]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, hidden_size, output_dim, dropout_rate):
        super(Perceptron, self).__init__()
        
        self.fc = nn.Linear(input_dim, hidden_size)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, output_dim)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        hidden = self.fc(x)
        relu = self.relu(hidden)
        relu = self.drop(relu)
        outputs = self.fc2(relu)
        outputs = self.sigmoid(outputs)
        return outputs

In [82]:
batch_size = 64
iterations = 5000
dropout_rate = 0.1
epochs = iterations / (len(train_dataset) / batch_size)

model = Perceptron(784, 64, 10, dropout_rate)

In [83]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
learning(criterion, optimizer)

Loss: 1.545954704284668. Accuracy: 93.


In [84]:
batch_size = 32
iterations = 4000
dropout_rate = 0.2
epochs = iterations / (len(train_dataset) / batch_size)

model = Perceptron(784, 64, 10, dropout_rate)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.001)
learning(criterion, optimizer)

Loss: 1.697817325592041. Accuracy: 80.


In [85]:
batch_size = 128
iterations = 3000
dropout_rate = 0.1
epochs = iterations / (len(train_dataset) / batch_size)

model = Perceptron(784, 64, 10, dropout_rate)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.8)
learning(criterion, optimizer)

Loss: 1.6763228178024292. Accuracy: 80.


## Выводы

Были обучены две модели - логистическая регрессия и перцептрон с разными гиперпараметрами. Результаты получились схожими по качеству и весьма неплохими в случае оптимайзера Adam, что объясняется качественными данными в датасете и подходящими для решения задачи методами. Использование большого dropout_rate и батча большого размера accuracy значительно упала в случае многослойного перцептрона